# App to find Cheap Flights

# Introduction:

In 2014, the cheapest fare from New York to Vienna was found to be around $800, but according to the advertised fares, where for a select no. of dates, these tickets were between $350 and $450. 

It all seemed to be a good deal and one might wonder if whether it is true or not. The industry does mistake the occasional mistakes on fares, because airlines occasionally and accidentally do happen to post fares that exclude fuel surcharges. Normally, it is expected that the advanced algorithms employed by these airlines would be updating fares that takes into account large number of factors, however due to the order generations of systems in place, mistakes do happen.



# 1 Import the necessary libraries:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 2 - Retrieving the Data from scraping the web:

Fare data are obtained from a AJAX-based (Asynchornous JavaScript) webpage, this will require a browser to do the work. For such a task, there will be a need for two of the following pacakges: Selenium and ChromeDriver.

- Selenium is a package for automating web browsers.
- ChromeDriver is a headless browser, meaning there isn't a user interface.

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver

chromeDriver_file = 'chromedriver'
chromeDriver_file_conda = 'chromedriver-binary alias'

import os
path = os.path.abspath(chromeDriver_file)
print('pathway to ChromeDriver is: ' + '\n' + path)

# Set the ChromeDriver pathway:
chromeDriver_path = path

browser = webdriver.Chrome(chromeDriver_path)

pathway to ChromeDriver is: 
/Users/y.s.lee/OneDrive/Packt - Python Machine Learning Blue Prints/Project 2 - App to find Cheap Flights/chromedriver


### 2.1 Set the URL (from google flights):

Dates are set to 1st of June to 15th of June in the year 2020 (note: that these dates can be changed to anything).

NOTE: Need to use the Freebase IDs for city/region of interest for travel. for example, m/06y57 is for Sydney. m/0f04v is for empty search. m/02_286 is for NYC.

It is possible to find it when searching in google from th ebelow link:
https://www.google.com.mx/travel/guide?q=New+York+City&sa=X&rlz=1C1CHBD_esMX769MX769&output=search&tra=%5B%22AMAbHIJDZRALeKKuHEbLXHGOJ3aS9zzCTg:1579328461567%22,%22syndey%22,%22/m/02_286%22%5D&tcfs=EhUKCS9tLzAyXzI4NhIITmV3IFlvcms&dest_mid=/m/06y57#dest_mid=/m/06y57&tcfs=EiwKCC9tLzA2eTU3EgZTeWRuZXkaGAoKMjAyMC0wMi0wMxIKMjAyMC0wMi0wNw

And to confirm it with the link below: make sure to control+f and search for Freebase ID.
https://www.wikidata.org/wiki/Q3130

In [3]:
# # Input webpage as string:
# flight_web_sats = 'https://www.google.com/travel/explore?tfs=CBsQAxojagcIARIDU1lEEgoyMDIwLTA2LTAxcgwIBBIIL20vMDJqOXoaI2oMCAQSCC9tLzAyajl6EgoyMDIwLTA2LTE1cgcIARIDU1lEcAFAAUgB&curr=USD&gl=us&hl=en&authuser=0&origin=https%3A%2F%2Fwww.google.com&dest_mid'
# # flight_web_sats = 'https://www.google.com/flights?hl=en#flt=/m/0f04v..2020-02-03*./m/0f04v.2020-02-07;c:USD;e:1;ls:1w;sd:0;t:h'

# # Retrieve the webpage's content using Selenium:
# browser.get(flight_web_sats)

# # Check the title of the webpage:
# browser.title

In [4]:
# # Check to see if the required information from the webpage was captured: Take a Screenshot and save as 'test_flights.png'.
# current_work_directory = os.getcwd()
# browser.save_screenshot(current_work_directory + '/test_flights.png')

### 2.2 Parsing the DOM to extract the individual flight data from the HTML tags:

Document Object Model (DOM) is the collection of the individual elements on a webpage. These will include things like HTML tags, like 'body' and 'div', or classes and IDs.

In [5]:
# # Parsing:
# soup = BeautifulSoup(browser.page_source, "html.parser")

#### Extract the individual city data:

In [6]:
# # Get the city data:
# # At the time of HTML scraping, the flight data was inside <div class='yP0obc'> or <li class='lPyEac'>

# # <div class="yP0obc" jscontroller="TYNAne" jsaction="pZvd5d:qA2ZTb;GbPxTb:WyQnIb;lxEPnd:uWrFbb;JIbuQc:NOeT2b(NOeT2b);"><div class="pWXB0b" jsname="aZ2wEe" style="display: none;"><div class="BngbEe"><div class="EmVfjc qs41qe" data-loadingmessage="Loading..." jscontroller="qAKInc" jsaction="animationend:kWijWc;dyRcpb:dyRcpb" data-active="true"><div class="Cg7hO" aria-live="assertive" jsname="vyyg5">Loading...</div><div jsname="Hxlbvc" class="xu46lf"><div class="ir3uv uWlRce co39ub"><div class="xq3j6 ERcjC"><div class="X6jHbb GOJTSe"></div></div><div class="HBnAAc"><div class="X6jHbb GOJTSe"></div></div><div class="xq3j6 dj3yTd"><div class="X6jHbb GOJTSe"></div></div></div><div class="ir3uv GFoASc Cn087"><div class="xq3j6 ERcjC"><div class="X6jHbb GOJTSe"></div></div><div class="HBnAAc"><div class="X6jHbb GOJTSe"></div></div><div class="xq3j6 dj3yTd"><div class="X6jHbb GOJTSe"></div></div></div><div class="ir3uv WpeOqd hfsr6b"><div class="xq3j6 ERcjC"><div class="X6jHbb GOJTSe"></div></div><div class="HBnAAc"><div class="X6jHbb GOJTSe"></div></div><div class="xq3j6 dj3yTd"><div class="X6jHbb GOJTSe"></div></div></div><div class="ir3uv rHV3jf EjXFBf"><div class="xq3j6 ERcjC"><div class="X6jHbb GOJTSe"></div></div><div class="HBnAAc"><div class="X6jHbb GOJTSe"></div></div><div class="xq3j6 dj3yTd"><div class="X6jHbb GOJTSe"></div></div></div></div></div></div></div><div jsname="bwERVd" data-ved="2ahUKEwj855qM9oznAhVO21sKHbmVA-MQ9IcBegQIARAX" jsaction="keydown:Uwbnrf"><ol class="SD4Ugf"><li class="lPyEac" data-code="/m/04jpl" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CAEQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click" tabindex="0"><div class="MebuN"><div class="L32YH " style="background-image: url('//t1.gstatic.com/images?q=tbn:ANd9GcSWD3RIhb5WVm8o0tVuh7Ygbe67MxTKwnYGRJ1cbhLQupYphfzlG37c6WbGvIIwlP7oGh1QoYZq'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">London</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="China Airlines" src="//www.gstatic.com/flights/airline_logos/70px/CI.png" data-iml="5379.940000000715"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 3 hr 20 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESATAaCwj+wAQQAhoDVVNE" class="QB2Jof xLPuCe">$739</span></div></div></div></div></li><li class="lPyEac" data-code="/m/05qtj" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CAIQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t0.gstatic.com/images?q=tbn:ANd9GcTjMNfNOfB7p7trIuVdo_2jYTPv5TmLw0QR-PETsSbud-DbJRlXlPPmszFepkr2R9b9ZKMbtyZl'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Paris</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="Vietnam Airlines" src="//www.gstatic.com/flights/airline_logos/70px/VN.png" data-iml="5378.889999999956"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 45 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESATEaCwjf4AQQAhoDVVNE" class="QB2Jof xLPuCe">$780</span></div></div></div></div></li><li class="lPyEac" data-code="/m/06c62" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CAMQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t0.gstatic.com/images?q=tbn:ANd9GcT8wlqMYPykCUfDKyo9qkeJ6ghDD6lwYjBDBPtefTUiwC1KxSQ1Kk_eBdABisyIN4WwkIOy5Oc'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Rome</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="China Airlines" src="//www.gstatic.com/flights/airline_logos/70px/CI.png" data-iml="5379.885000000286"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 17 hr 35 min</span></div></div><div class="Q70fcd sSHqwe"><div class="wRabvc">$369 less than usual</div><div class="MJg7fb"><span class="ZFJngd bKyNQc">Great value</span><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESATIaCwiz5AMQAhoDVVNE" class="QB2Jof xLPuCe OKVlp">$621</span></div></div></div></div></li><li class="lPyEac" data-code="/m/0k3p" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CAQQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t1.gstatic.com/images?q=tbn:ANd9GcQr7zHh49pOLCaYkz_EMohL9YqtqN21ach939W0YODvNyn5ICUn2j7NX3HqQyGzBmnT823MlhI'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Amsterdam</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="China Airlines" src="//www.gstatic.com/flights/airline_logos/70px/CI.png" data-iml="5380.040000000008"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 17 hr 30 min</span></div></div><div class="Q70fcd sSHqwe"><div class="wRabvc">$373 less than usual</div><div class="MJg7fb"><span class="ZFJngd bKyNQc">Great value</span><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESATMaCwjQ5wMQAhoDVVNE" class="QB2Jof xLPuCe OKVlp">$625</span></div></div></div></div></li><li class="lPyEac" data-code="/m/0n2z" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CAUQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t0.gstatic.com/images?q=tbn:ANd9GcSi0fVq1M5hNL0E1hNfPcujD_u1MIfur58_HJn7tHq6bM-7UJ-14ltRCa_A8DXLbbdXs64UqsCo'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Athens</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="Alitalia, China Airlines, and Scoot" src="//www.gstatic.com/flights/airline_logos/70px/multi.png" data-iml="5378.349999999045"><span class="nx0jzf">2 stops</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 3 hr 20 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESATQaCwjtlwUQAhoDVVNE" class="QB2Jof xLPuCe">$850</span></div></div></div></div></li><li class="lPyEac" data-code="/m/02cft" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CAYQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t2.gstatic.com/images?q=tbn:ANd9GcThSCLQVoapbqAOX1UpCL3j4JPptKaog6nD9XGS59u8WYslqwI3GvGbzmKjUYVihl3TrQyBWkA'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Dublin</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="EK" src="//www.gstatic.com/flights/airline_logos/70px/EK.png" data-iml="5379.605000000083"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">23 hr 10 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESATUaCwjwqwUQAhoDVVNE" class="QB2Jof xLPuCe">$876</span></div></div></div></div></li><li class="lPyEac" data-code="/m/052bw" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CAcQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t1.gstatic.com/images?q=tbn:ANd9GcS7P6NanAHr-ksMXhE4AoZXFzuwipPuXs9HcjHMf__PG5CBlGcszYJEwz511VbHBVK2bHXmSnQ'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Manchester</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="SWISS and Singapore Airlines" src="//www.gstatic.com/flights/airline_logos/70px/multi.png" data-iml="5377.324999999473"><span class="nx0jzf">2 stops</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 6 hr 5 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESATYaCwidtwUQAhoDVVNE" class="QB2Jof xLPuCe">$890</span></div></div></div></div></li><li class="lPyEac" data-code="/m/02z0j" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CAgQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t3.gstatic.com/images?q=tbn:ANd9GcQAXxjuz4bgYB9Z5hKKCMwryC8QsQ9xMh13dQfjanUDnRvdOcjY51DeWk7lXr0BdGr1t1nPvFU'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Frankfurt</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="China Airlines" src="//www.gstatic.com/flights/airline_logos/70px/CI.png" data-iml="5379.989999999452"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 16 hr 40 min</span></div></div><div class="Q70fcd sSHqwe"><div class="wRabvc">$336 less than usual</div><div class="MJg7fb"><span class="ZFJngd bKyNQc">Great value</span><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESATcaCwjpmQQQAhoDVVNE" class="QB2Jof xLPuCe OKVlp">$689</span></div></div></div></div></li><li class="lPyEac" data-code="/m/09949m" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CAkQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t2.gstatic.com/images?q=tbn:ANd9GcRNudOo_G9Mgl435C2PDDnk8_mzPX3DP_MStETEDseiXoVlhKF3qJfqmdfMXaLxW7MJhqYbhEBX'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">İstanbul</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="China Southern" src="//www.gstatic.com/flights/airline_logos/70px/CZ.png" data-iml="5380.165000000488"><span class="nx0jzf">2 stops</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">2 days 17 hr 55 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESATgaCwjbpQUQAhoDVVNE" class="QB2Jof xLPuCe">$868</span></div></div></div></div></li><li class="lPyEac" data-code="/m/06mxs" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CAoQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t0.gstatic.com/images?q=tbn:ANd9GcTqRjj2iLbCubX30fPGdMdrdBKVaY3K8maDs-cduFBaoSwBjwwkSA4y_ujDdhHujrk9a5Wb0-pa'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Stockholm</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="EK" src="//www.gstatic.com/flights/airline_logos/70px/EK.png" data-iml="5379.754999999932"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 5 hr 5 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESATkaCwjgkwYQAhoDVVNE" class="QB2Jof xLPuCe">$1,009</span></div></div></div></div></li><li class="lPyEac" data-code="/m/01f62" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CAsQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t0.gstatic.com/images?q=tbn:ANd9GcQbk5myJfeEO1Fz1KzETkT7DoiXFglEUDWHNMYSV49AYMD72zGpEgtAvserKOgXLwkygy67jds'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Barcelona</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="Lufthansa, SWISS, and Singapore Airlines" src="//www.gstatic.com/flights/airline_logos/70px/multi.png" data-iml="5378.469999999652"><span class="nx0jzf">2 stops</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 5 hr 35 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjEwGgsI0uYEEAIaA1VTRA==" class="QB2Jof xLPuCe">$787</span></div></div></div></div></li><li class="lPyEac" data-code="/m/0947l" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CAwQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t1.gstatic.com/images?q=tbn:ANd9GcS5cdz6GSfEry8guAX0jZY5Ly9eKhDdD9ecBPl1qxOs9eLuVppxiYBaHkL1z_9WxJRQzUdDmrQ'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Milan</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="Cathay Pacific" src="//www.gstatic.com/flights/airline_logos/70px/CX.png" data-iml="5379.12999999935"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 1 hr 15 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjExGgsIrJ4FEAIaA1VTRA==" class="QB2Jof xLPuCe">$859</span></div></div></div></div></li><li class="lPyEac" data-code="/m/0156q" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CA0QxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t3.gstatic.com/images?q=tbn:ANd9GcQoh6NRGCaPTWQU3o0oB8XkSh3D4hmRdHq3odk-SA6I4Gl4dKdvl_QQcXNhaca_rvjQ47VmAHg2'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Berlin</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="Scoot" src="//www.gstatic.com/flights/airline_logos/70px/TR.png" data-iml="5379.54499999978"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 18 hr 35 min</span></div></div><div class="Q70fcd sSHqwe"><div class="wRabvc">$364 less than usual</div><div class="MJg7fb"><span class="ZFJngd bKyNQc">Great value</span><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjEyGgsIyIcEEAIaA1VTRA==" class="QB2Jof xLPuCe OKVlp">$666</span></div></div></div></div></li><li class="lPyEac" data-code="/m/056_y" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CA4QxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t3.gstatic.com/images?q=tbn:ANd9GcT5byDr37DEdaUNB2LskpTCgCnd-wTrrAAa1xvr5TiZTsgdeKdNhknLG8d_17vF8WH4nsthN4U'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Madrid</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="Cathay Pacific" src="//www.gstatic.com/flights/airline_logos/70px/CX.png" data-iml="5379.044999999678"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 6 hr 45 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjEzGgsIlJ0FEAIaA1VTRA==" class="QB2Jof xLPuCe">$857</span></div></div></div></div></li><li class="lPyEac" data-code="/m/08966" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CA8QxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t3.gstatic.com/images?q=tbn:ANd9GcQrbJo33WQCWED4UTbpmou6ffwZk4qfxupyDYkjb46WNg9McDfc3iZSJrDqIhm7NLrlRmEXY26-'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Zürich</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="Cathay Pacific" src="//www.gstatic.com/flights/airline_logos/70px/CX.png" data-iml="5379.194999999527"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 15 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjE0GgsIkaMFEAIaA1VTRA==" class="QB2Jof xLPuCe">$865</span></div></div></div></div></li><li class="lPyEac" data-code="/m/02h6_6p" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CBAQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t3.gstatic.com/images?q=tbn:ANd9GcTLDeDJj8aQuVOtnMy68xRWogm9tJOKgHPkYH1i5F0BHXzOFnbJWSzDDxvieoXDiGOVWuu1kpRP'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Munich</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="Etihad" src="//www.gstatic.com/flights/airline_logos/70px/EY.png" data-iml="5370.005000000674"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">23 hr</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjE1GgsIx+4FEAIaA1VTRA==" class="QB2Jof xLPuCe">$961</span></div></div></div></div></li><li class="lPyEac" data-code="/m/01lfy" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CBEQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t3.gstatic.com/images?q=tbn:ANd9GcQpyktCK0en2fRyOBL61pUNrg4S63iWUnQ7IAffeY2wdOF-evlkNZkj6g_QfZQZhSTgfitgfqlm'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Copenhagen</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="China Southern and KLM" src="//www.gstatic.com/flights/airline_logos/70px/multi.png" data-iml="5378.044999999474"><span class="nx0jzf">2 stops</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 9 hr</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjE2GgsIhO4EEAIaA1VTRA==" class="QB2Jof xLPuCe">$797</span></div></div></div></div></li><li class="lPyEac" data-code="/m/06flg" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CBIQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t1.gstatic.com/images?q=tbn:ANd9GcR8ka5LwAWVsW_eL-mFe8nqDEYuUY9BO85AtnpmTPc-CiXxK0wt24y4jJYuR-r7Guc_BmHy9O6e'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Reykjavík</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="Finnair and Qantas" src="//www.gstatic.com/flights/airline_logos/70px/multi.png" data-iml="5377.6099999995495"><span class="nx0jzf">2 stops</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 5 hr 35 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjE3GgsIscEHEAIaA1VTRA==" class="QB2Jof xLPuCe">$1,231</span></div></div></div></div></li><li class="lPyEac" data-code="/m/0fhp9" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CBMQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t3.gstatic.com/images?q=tbn:ANd9GcQ6TxY5kOvK30f7EO4PbwEF88N04mRV7hneAtVQukFCZ5qWSr7ids5cWIyuyw2MLHxKiEDh2Bxu'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Vienna</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="Austrian and TG" src="//www.gstatic.com/flights/airline_logos/70px/multi.png" data-iml="5377.809999999954"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 3 hr 35 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjE4GgsIwdYFEAIaA1VTRA==" class="QB2Jof xLPuCe">$930</span></div></div></div></div></li><li class="lPyEac" data-code="/m/02m77" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CBQQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t3.gstatic.com/images?q=tbn:ANd9GcQb_Kl9VzrT6FiDnv8k96ZEMLLMt4BwMS5H7VkUllgew4EBTrP_7fBoQRKczDFErDWMHpPCe6-Y'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Edinburgh</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="EK" src="//www.gstatic.com/flights/airline_logos/70px/EK.png" data-iml="5379.70999999925"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 7 hr 45 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjE5GgsIx8AFEAIaA1VTRA==" class="QB2Jof xLPuCe">$902</span></div></div></div></div></li><li class="lPyEac" data-code="/m/04swd" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CBUQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t3.gstatic.com/images?q=tbn:ANd9GcTvAdSnob5sZvgzpDvwWbv1swIPUIsRK3QxX51lReAFPlPe2wcGOOT17YYi3xm4G7gwR_G0RMD9'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Moscow</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="China Southern" src="//www.gstatic.com/flights/airline_logos/70px/CZ.png" data-iml="5380.105000000185"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 16 hr 10 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjIwGgsI+64FEAIaA1VTRA==" class="QB2Jof xLPuCe">$880</span></div></div></div></div></li><li class="lPyEac" data-code="/m/04v3q" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CBYQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t2.gstatic.com/images?q=tbn:ANd9GcSjZIc4mJeM8eWY8CKtreGM0eI3agtpOxznINw-kaxAPKeHJFwnUXMTL4rZe8Szr7af7Ih99HI'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Malta</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="Qatar Airways" src="//www.gstatic.com/flights/airline_logos/70px/QR.png" data-iml="5379.259999999704"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 10 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjIxGgsIiPAFEAIaA1VTRA==" class="QB2Jof xLPuCe">$963</span></div></div></div></div></li><li class="lPyEac" data-code="/m/03khn" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CBcQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t0.gstatic.com/images?q=tbn:ANd9GcTEQ-ZO8AxWCNZ9oS0zrlyiugotgf3mw_PX8a9jbxpzMdKk90Ta7XJZPoYypGjmppr1UhpKbOM'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Helsinki</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="Finnair and Qantas" src="//www.gstatic.com/flights/airline_logos/70px/multi.png" data-iml="5377.6899999993475"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">21 hr 20 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjIyGgsI67YGEAIaA1VTRA==" class="QB2Jof xLPuCe">$1,054</span></div></div></div></div></li><li class="lPyEac" data-code="/m/07_pf" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CBgQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t0.gstatic.com/images?q=tbn:ANd9GcTPxb09v-NeA5bPrySFJw-mPjSNx_IKRAhFwEYjJT9f1hQJsLIiGZHrQYdT4DPPRO0rv_FrDEw'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Venice</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="British Airways, SWISS, and Singapore Airlines" src="//www.gstatic.com/flights/airline_logos/70px/multi.png" data-iml="5378.409999999349"><span class="nx0jzf">2 stops</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 1 hr 50 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjIzGgsI0KEGEAIaA1VTRA==" class="QB2Jof xLPuCe">$1,027</span></div></div></div></div></li><li class="lPyEac" data-code="/m/095w_" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CBkQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t2.gstatic.com/images?q=tbn:ANd9GcQLotaanX1qD_U695g2PcVwcLlhNZou4rjnNlwZqoVqOU8cueG11ysnl5oAVgS6eAVeaCLlqcF9'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Budapest</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="British Airways" src="//www.gstatic.com/flights/airline_logos/70px/BA.png" data-iml="5380.424999999377"><span class="nx0jzf">2 stops</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 4 hr 15 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjI0GgsI7fAFEAIaA1VTRA==" class="QB2Jof xLPuCe">$964</span></div></div></div></div></li><li class="lPyEac" data-code="/m/03902" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CBoQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t0.gstatic.com/images?q=tbn:ANd9GcQ08qiEY58CdHe6ruw8Ywm3-CnP8B0KI37rM5h7UjHNQOPLptiZSxs1NuEUu6y5U--Ji_PtJAL3'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Geneva</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="British Airways" src="//www.gstatic.com/flights/airline_logos/70px/BA.png" data-iml="5380.470000000059"><span class="nx0jzf">2 stops</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 2 hr 30 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjI1GgsIiewFEAIaA1VTRA==" class="QB2Jof xLPuCe">$958</span></div></div></div></div></li><li class="lPyEac" data-code="/m/04llb" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CBsQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t0.gstatic.com/images?q=tbn:ANd9GcQVTgrePbKKZRuJKVfV7liPd1R5b8uzlL1qASGvH-0H-MP-G7-lzOc5Iu4hRTMkh2azrhErNd4'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Lisbon</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="Qatar Airways" src="//www.gstatic.com/flights/airline_logos/70px/QR.png" data-iml="5379.370000000563"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 8 hr 10 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjI2GgsIodcFEAIaA1VTRA==" class="QB2Jof xLPuCe">$931</span></div></div></div></div></li><li class="lPyEac" data-code="/m/0fhzf" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CBwQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t0.gstatic.com/images?q=tbn:ANd9GcQfOG_7U-nidYwhLvuNwUFsNyY0VFnj9oHEj6IrrsGjwy60DRd-Z1PnfzK7-AjY7uhOL_xl4y3Q'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Belgrade</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="Qatar Airways" src="//www.gstatic.com/flights/airline_logos/70px/QR.png" data-iml="5379.31000000026"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">21 hr 50 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjI3GgsIiPUFEAIaA1VTRA==" class="QB2Jof xLPuCe">$970</span></div></div></div></div></li><li class="lPyEac" data-code="/m/02rnbv" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CB0QxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t1.gstatic.com/images?q=tbn:ANd9GcTd7eexz9nS94zQ8By-fQVsgOck_XICqgd92O6QWYBek6LvDcMuryO3JKe1YG1G59wYEpmdi3U'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Brussels</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="Etihad" src="//www.gstatic.com/flights/airline_logos/70px/EY.png" data-iml="5370.215000000826"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">23 hr 25 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjI4GgsI+7oFEAIaA1VTRA==" class="QB2Jof xLPuCe">$895</span></div></div></div></div></li><li class="lPyEac" data-code="/m/05ywg" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CB4QxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t0.gstatic.com/images?q=tbn:ANd9GcRe5fsY8ckriJwYLCytZfDH_APYzNK2l67NI3mIgei6xcN16CWgAFfBxtuCvG29DQhTIZ0vI4g'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Prague</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="British Airways" src="//www.gstatic.com/flights/airline_logos/70px/BA.png" data-iml="5380.535000000236"><span class="nx0jzf">2 stops</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 2 hr 45 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjI5GgsIrOoFEAIaA1VTRA==" class="QB2Jof xLPuCe">$956</span></div></div></div></div></li><li class="lPyEac" data-code="/m/07bbc" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CB8QxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t1.gstatic.com/images?q=tbn:ANd9GcSQ2H0iRPcau2EXitLQI3oN8nA8OJ1UwheMcLbVtfNUJUIHQM1AQ-hhLfyBtDobLcGM-dTm-YDF'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Skopje</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="Austrian, Singapore Airlines, and TG" src="//www.gstatic.com/flights/airline_logos/70px/multi.png" data-iml="5378.2599999995"><span class="nx0jzf">3 stops</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 9 hr 50 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjMwGgsIjvQFEAIaA1VTRA==" class="QB2Jof xLPuCe">$968</span></div></div></div></div></li><li class="lPyEac" data-code="/m/05l64" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CCAQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t3.gstatic.com/images?q=tbn:ANd9GcQR3BE9WdtTBQ9rAhJhjguQPxmo5MP6-k0V4Ecm7jqQN4R34TvxeQNh_O7D8X-Y-FhOajnmw2YU'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Oslo</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="Qatar Airways" src="//www.gstatic.com/flights/airline_logos/70px/QR.png" data-iml="5379.4199999993"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjMxGgsImJ8GEAIaA1VTRA==" class="QB2Jof xLPuCe">$1,023</span></div></div></div></div></li><li class="lPyEac" data-code="/m/081m_" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CCEQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t3.gstatic.com/images?q=tbn:ANd9GcTajcPTou5BsXTNdYBsQzYWapNZbm33hmwjmKZn8oZlF7A-lb7XplxKKdiGJu1Wk1IxO4qeFWY'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Warsaw</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="EK" src="//www.gstatic.com/flights/airline_logos/70px/EK.png" data-iml="5379.820000000109"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 4 hr 10 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjMyGgsIg5IGEAIaA1VTRA==" class="QB2Jof xLPuCe">$1,007</span></div></div></div></div></li><li class="lPyEac" data-code="/m/0hyxv" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CCIQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t2.gstatic.com/images?q=tbn:ANd9GcRnYZqOooVbXnPHOMnV5Hd0HvAaWbry-XV45z2_EgqhYiAqA6twFfU8O6qLpnk26yQmJZGt24Y'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Glasgow</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="British Airways" src="//www.gstatic.com/flights/airline_logos/70px/BA.png" data-iml="5380.329999999958"><span class="nx0jzf">2 stops</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 2 hr 45 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjMzGgsImtAFEAIaA1VTRA==" class="QB2Jof xLPuCe">$922</span></div></div></div></div></li><li class="lPyEac" data-code="/m/0k33p" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CCMQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t3.gstatic.com/images?q=tbn:ANd9GcQ6XSrEMJPOFqZfv1gq8ApwyvncuYpuX39W6YwKiqs4kNIfi6eBIfQ3u8OcLN4bNpdfyTBJV3ve'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Birmingham</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="EK" src="//www.gstatic.com/flights/airline_logos/70px/EK.png" data-iml="5379.660000000513"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 5 hr 15 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjM0GgsIrJYGEAIaA1VTRA==" class="QB2Jof xLPuCe">$1,012</span></div></div></div></div></li><li class="lPyEac" data-code="/m/0cp6w" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CCQQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t0.gstatic.com/images?q=tbn:ANd9GcQhRl_jY7sbmmhEqzl9YhiGzM2GSFtylyTF4GbJA9NQLsaYVkRIXKsOTLjz6N5t716pEL2BWgg'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Nice</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="British Airways" src="//www.gstatic.com/flights/airline_logos/70px/BA.png" data-iml="5380.225000000792"><span class="nx0jzf">2 stops</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 4 hr 5 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjM1GgsIjPIFEAIaA1VTRA==" class="QB2Jof xLPuCe">$966</span></div></div></div></div></li><li class="lPyEac" data-code="/m/0fhzy" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CCUQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t1.gstatic.com/images?q=tbn:ANd9GcRCZBomjzLC5pbYoNO6pPiJ-fmoDccjiS8eRt-bEauDaIxObcG87kESd1akqBJrInqfKovCtOo'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Zagreb</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="Qatar Airways" src="//www.gstatic.com/flights/airline_logos/70px/QR.png" data-iml="5379.484999999477"><span class="nx0jzf">1 stop</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">23 hr 55 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjM2GgsI/5MGEAIaA1VTRA==" class="QB2Jof xLPuCe">$1,009</span></div></div></div></div></li><li class="lPyEac" data-code="/m/0hf7l" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CCYQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t0.gstatic.com/images?q=tbn:ANd9GcR7WTq3IzvcgB381sLyBMeNCo7ehtmSPgI9OfXNlT5bGyfErSymz465kpDJqmF_PZvGxwl7lSU'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Düsseldorf</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="Lufthansa and Singapore Airlines" src="//www.gstatic.com/flights/airline_logos/70px/multi.png" data-iml="5377.744999999777"><span class="nx0jzf">2 stops</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">23 hr 30 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjM3GgsIsowGEAIaA1VTRA==" class="QB2Jof xLPuCe">$999</span></div></div></div></div></li><li class="lPyEac" data-code="/m/03hrz" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CCcQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t2.gstatic.com/images?q=tbn:ANd9GcRVkrmNIHq_6KxvemwifpgrKQdaKKAXMmnJcK7jGojpjWt8ht81aX3pJ63eNptPp-KmnTqRu9ig'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Hamburg</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="British Airways" src="//www.gstatic.com/flights/airline_logos/70px/BA.png" data-iml="5380.37500000064"><span class="nx0jzf">2 stops</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 3 hr</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjM4GgsIx5gGEAIaA1VTRA==" class="QB2Jof xLPuCe">$1,015</span></div></div></div></div></li><li class="lPyEac" data-code="/m/0j7ng" role="button" jsaction="click:Qpi0ec; mouseenter:HvNCCb; mouseleave:uGBZrc" jslog="14149;ved:0CCgQxW5qFwoTCPjRpI72jOcCFQAAAAAdAAAAABAF;track:click"><div class="MebuN"><div class="L32YH " style="background-image: url('//t2.gstatic.com/images?q=tbn:ANd9GcQxHn8Bq6LhOFVxx5kOrgcNlVN2-NgCIaLHWmWTj6PdQAbCnW68g9LTK3HJD-OsNUjCj3U78GM'), url('//www.gstatic.com/flights/app/runway_200.png')"></div><div class="tsAU4e "><div class="wIuJz"><h3 class="W6bZuc YMlIz">Newcastle upon Tyne</h3><div class="ZjDced CQYfx"><img class="C5fbBf" height="16" width="16" alt="British Airways" src="//www.gstatic.com/flights/airline_logos/70px/BA.png" data-iml="5380.2749999995285"><span class="nx0jzf">2 stops</span><span class="qeoz6e U325Rc"></span><span class="Xq1DAb">1 day 2 hr 20 min</span></div></div><div class="Q70fcd sSHqwe"><div class="MJg7fb"><span data-gs="CidHOXd0X0JHLS0tLS0tLS1wZnd3MTI0QUFBQUFGNGkyMzhCVXJzQUESAjM5GgsIvtMFEAIaA1VTRA==" class="QB2Jof xLPuCe">$927</span></div></div></div></div></li></ol></div></div>


# flight_cards = soup.select('div[class*=yP0obc]')

# # flight_cards = soup.select('div[class*=yP0obc]')

# # Check out a single flight card:
# flight_cards[0]


From the piece of HTML information above, it can be noticed that the information needed is within the markup. 
For example: 
- the destination is seen here 'class="W6bZuc YMlIz" London',
- where the duration of the flight is 'class="Xq1DAb" 1 day 3 hr 20 min' 
- and prices are located at 'class="QB2Jof xLPuCe" datags="CidHQ2lQekJHLS0tLS0tLS0tcGZiMTI5QUFBQUFGNGlrY29Bby1aQUESATAaCwj+wAQQAhoDVVNE">$739'.

#### Next, is to obtain the required data from the markup:


In [7]:
# # For-loop to extract the relevant information:
# # At the time of HTML scraping, the price information are stored in <div class='bwERVd'>

# # for card in flight_cards:
# #     print(card.select('h3')[0].text)
# #     print(card.select('div[class*=MJg7fb]')[0].text)
# #     print('\n')

# # Perform clean up of 'Great value' tags before the prices:
# for card in flight_cards:
#     print(card.select('h3')[0].text)
#     print(card.select('div[class*=bwERVd]')[0].text.replace('Great value',""))
#     print('\n')


From the above, it can be confirmed that it is possible to retrieve the relevant data from the HTML. 

#### Next, is to retrieve flights that presents with the lowest cost and non-stop fares from the starting destination to the arrival destination. These flights would be for a 26 week period. All this is done by making a full scrape and parsing of a large number of fares.

#### Import the required libraries:

In [8]:
import datetime
from datetime import date, timedelta
from time import sleep

In [18]:
# 
week_period = 26
start_date = '2020-06-01'
end_date = '2020-06-15'

# Format the flight dates: with the python datetime standard.
startFlight_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
endFlight_date = datetime.datetime.strptime(end_date, '%Y-%m-%d')

flightFare_dict = {}


sat_start = str(startFlight_date).split()[0]
sat_end = str(endFlight_date).split()[0]
# sats = "https://www.google.com/flights?hl=en#flt=/m/0f04v.." + sat_start + "*./m/0f04v." + sat_end + ";c:USD;e:1;ls:1w;sd:0;t:h"

# browser.get(sats)

# sats_2 = "https://www.google.com/flights/f=0#f=0&flt=/m/02_286.r/m/02j9z." + sat_start + "*r/m/02j9z./m/02_286." + sat_end + ";c:USD;e:1;ls:1w;sd:0;t:h"

sats_2 = "https://www.google.com/flights/f=0#f=0&flt=/m/06y57.r/m/02j9z." + sat_start + "*r/m/06y57./m/02_286." + sat_end + ";c:USD;e:1;ls:1w;sd:0;t:h"



# 'https://www.google.com/flights/f=0#f=0&flt=/m/02_286.r/m/02j9z.2018-12-01*r/m/02j9z./m/02_286.2018-12-08;c:USD;e:1;s:0*1;sd:1;t:e'
# "m/06y57", ""
browser.get(sats_2)

#===== click the website 'search' link ======
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions

# <floating-action-button class="gws-flights-form__search-button flt-form-sb gws-flights-fab__mini" data-flt-ve="search_button" jsaction="jsl._" role="button" jsan="7.gws-flights-form__search-button,7.flt-form-sb,0.tabindex,0.data-flt-ve,22.jsaction" jstcache="841" tabindex="0"><img class="flt-fab-icon" src="//ssl.gstatic.com/images/icons/material/system/2x/search_white_24dp.png" alt="" height="24" width="24" data-atf="1"><span class="gws-flights-fab__text">Search</span></floating-action-button>
# driver.find_elements_by_xpath("//button[@title='Save and Continue Edit']")



# link = browser.find_element_by_xpath('//floating-action-button[@title="Search"]')
# link = browser.find_element_by_xpath('//floating-action-button[class="gws-flights-form__search-button flt-form-sb gws-flights-fab__mini" data-flt-ve="search_button"]')

link = WebDriverWait(browser, 3).until(expected_conditions.text_to_be_present_in_element(By.ID, 'Search')



# link = browser.find_element_by_xpath('//span[contains(text(),"Search")]')
link.click()

#==== Save the new (current) URL, after clicking on search ====
# current_URL = browser.current_url
# print('The current (new) URL after clicking is: \n' + current_URL)


# for idx in range(week_period):
#     sat_start = str(startFlight_date).split()[0]
#     sat_end = str(endFlight_date).split()[0]
    
#     flightFare_dict.update({sat_start: {}})
#     sats = "https://www.google.com/flights?hl=en#flt=/m/0f04v.." + sat_start + "*./m/0f04v." + sat_end + ";c:USD;e:1;ls:1w;sd:0;t:h"
#     sleep(np.random.randint(3,7))
    
#     browser.get(sats)
#     soup = BeautifulSoup(browser.page_source, 'html.parser')
#     flight_cards = soup.select('div[class*=MebuN]')
#     for card in flight_cards:
#         city = card.select('h3')[0].text
#         print(city)
#         fare = card.select('div[class*=MJg7fb]')[0].text.replace('Great value',"")
#         print(fare)
#         flightFare_dict[sat_start] = {**flightFare_dict[sat_start], **{city: fare}}
        
#     startFlight_date = startFlight_date + timedelta(days = 7)
#     endFlight_date = endFlight_date + timedelta(days = 7)


SyntaxError: invalid syntax (<ipython-input-18-815d7b567998>, line 47)

In [ ]:
# https://www.google.com/flights?hl=en#flt=/m/0f04v..2020-02-03*./m/0f04v.2020-02-07;c:USD;e:1;ls:1w;sd:0;t:h

In [ ]:
# https://www.google.com/flights?hl=en#flt=/m/0f04v..2020-06-01*./m/0f04v.2020-06-15;c:USD;e:1;ls:1w;sd:0;t:h